In [ ]:
#@title Settings Select

league = 'IYNFYL'
year = 2021
week = 17

inputemail = ''
inputpassword = ''

In [ ]:
#@title Imports
%%capture
# !pip install selenium
# !apt-get update 
# !apt install chromium-chromedriver
# !pip install ff_espn_api;

import pandas as pd
from sklearn import linear_model
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from ff_espn_api import League
from itertools import chain,cycle
from IPython.display import display_html

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

df = pd.DataFrame()

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [ ]:
#@title Define lists and dicts

proj_column_dict = {
  2:['PlayerTeam','PlayerPosition','Week','OppTeam','ProjPassingCompletion','ProjPassingAttempts','ProjPassingPercent','ProjPassingYards','ProjPassingAverage','ProjPassingTouchdowns','ProjPassingInterceptions','ProjPassingRating','ProjRushingAttempts','ProjRushingYards','ProjRushingAverage','ProjRushingTouchdowns','ProjFPTS/G','ProjFantasyPoints'],
  3:['PlayerTeam','PlayerPosition','Week','OppTeam','ProjRushingAttempts','ProjRushingYards','ProjRushingAverage','ProjRushingTouchdowns','ProjReceivingTargets','ProjReceivingReceptions','ProjReceivingYards','ProjReceivingTouchdowns','ProjFumbles','ProjFumblesLost','ProjFPTS/G','ProjFantasyPoints'],
  4:['PlayerTeam','PlayerPosition','Week','OppTeam','ProjReceivingTargets','ProjReceivingReceptions','ProjReceivingPercent','ProjReceivingYards','ProjReceivingTouchdowns','ProjReceivingLong','ProjReceivingYards/Target','ProjReceivingYards/Reception','ProjRushingAttempts','ProjRushingYards','ProjRushingAverage','ProjRushingTouchdowns','ProjFumbles','ProjFumblesLost','ProjFPTS/G','ProjFantasyPoints'],
  5:['PlayerTeam','PlayerPosition','Week','OppTeam','ProjReceivingTargets','ProjReceivingReceptions','ProjReceivingPercent','ProjReceivingYards','ProjReceivingTouchdowns','ProjReceivingLong','ProjReceivingYards/Target','ProjReceivingYards/Reception','ProjRushingAttempts','ProjRushingYards','ProjRushingAverage','ProjRushingTouchdowns','ProjFumbles','ProjFumblesLost','ProjFPTS/G','ProjFantasyPoints'],
  6:['PlayerTeam','PlayerPosition','Week','OppTeam','ProjFieldGoalsMade','ProjFieldGoalAttempts','ProjFieldGoalsPercent','ProjFieldGoalsLong','ProjExtraPointsMade','ProjExtraPointAttempts','ProjFPTS/G','ProjFantasyPoints'],
  7:['PlayerTeam','PlayerPosition','Week','OppTeam','ProjTacklesForLoss','ProjSacks','ProjQuarterbackHits','ProjInterceptions','ProjFumblesRecovered','ProjSafeties','ProjDefensiveTouchdowns','ProjReturningTouchdowns','ProjPointsAllowed','ProjFPTS/G','ProjFantasyPoints'],
  }

perf_column_dict = {
  2:['PlayerTeam','PlayerPosition','Week','OppTeam','PerfPassingCompletion','PerfPassingAttempts','PerfPassingPercent','PerfPassingYards','PerfPassingAverage','PerfPassingTouchdowns','PerfPassingInterceptions','PerfPassingRating','PerfRushingAttempts','PerfRushingYards','PerfRushingAverage','PerfRushingTouchdowns','PerfFPTS/G','PerfFantasyPoints'],
  3:['PlayerTeam','PlayerPosition','Week','OppTeam','PerfRushingAttempts','PerfRushingYards','PerfRushingAverage','PerfRushingTouchdowns','PerfReceivingTargets','PerfReceivingReceptions','PerfReceivingYards','PerfReceivingTouchdowns','PerfFumbles','PerfFumblesLost','PerfFPTS/G','PerfFantasyPoints'],
  4:['PlayerTeam','PlayerPosition','Week','OppTeam','PerfReceivingTargets','PerfReceivingReceptions','PerfReceivingPercent','PerfReceivingYards','PerfReceivingTouchdowns','PerfReceivingLong','PerfReceivingYards/Target','PerfReceivingYards/Reception','PerfRushingAttempts','PerfRushingYards','PerfRushingAverage','PerfRushingTouchdowns','PerfFumbles','PerfFumblesLost','PerfFPTS/G','PerfFantasyPoints'],
  5:['PlayerTeam','PlayerPosition','Week','OppTeam','PerfReceivingTargets','PerfReceivingReceptions','PerfReceivingPercent','PerfReceivingYards','PerfReceivingTouchdowns','PerfReceivingLong','PerfReceivingYards/Target','PerfReceivingYards/Reception','PerfRushingAttempts','PerfRushingYards','PerfRushingAverage','PerfRushingTouchdowns','PerfFumbles','PerfFumblesLost','PerfFPTS/G','PerfFantasyPoints'],
  6:['PlayerTeam','PlayerPosition','Week','OppTeam','PerfFieldGoalsMade','PerfFieldGoalAttempts','PerfFieldGoalsPercent','PerfFieldGoalsLong','PerfExtraPointsMade','PerfExtraPointAttempts','PerfFPTS/G','PerfFantasyPoints'],
  7:['PlayerTeam','PlayerPosition','Week','OppTeam','PerfTacklesForLoss','PerfSacks','PerfQuarterbackHits','PerfInterceptions','PerfFumblesRecovered','PerfSafeties','PerfDefensiveTouchdowns','PerfReturningTouchdowns','PerfPointsAllowed','PerfFPTS/G','PerfFantasyPoints'],
  }

relevant_data_columns = [
  'Year','Week', 'PlayerPosition', 'OppTeam',
  'ProjPassingCompletion', 'ProjPassingAttempts', 'ProjPassingYards', 'ProjPassingTouchdowns', 'ProjPassingInterceptions', 'ProjRushingAttempts', 'ProjRushingYards', 'ProjRushingTouchdowns', 'ProjReceivingTargets', 'ProjReceivingReceptions', 'ProjReceivingYards', 'ProjReceivingTouchdowns', 'ProjFumbles', 'ProjFumblesLost', 'ProjFieldGoalsMade', 'ProjFieldGoalAttempts', 'ProjExtraPointsMade', 'ProjExtraPointAttempts',
  'PerfPassingCompletion', 'PerfPassingAttempts', 'PerfPassingYards', 'PerfPassingTouchdowns', 'PerfPassingInterceptions', 'PerfRushingAttempts', 'PerfRushingYards', 'PerfRushingTouchdowns', 'PerfReceivingTargets', 'PerfReceivingReceptions', 'PerfReceivingYards', 'PerfReceivingTouchdowns', 'PerfFumbles', 'PerfFumblesLost', 'PerfFieldGoalsMade', 'PerfFieldGoalAttempts', 'PerfExtraPointsMade', 'PerfExtraPointAttempts',
  'ProjTacklesForLoss','ProjSacks','ProjQuarterbackHits','ProjInterceptions','ProjFumblesRecovered','ProjSafeties','ProjDefensiveTouchdowns','ProjReturningTouchdowns','ProjPointsAllowed',
  'PerfTacklesForLoss','PerfSacks','PerfQuarterbackHits','PerfInterceptions','PerfFumblesRecovered','PerfSafeties','PerfDefensiveTouchdowns','PerfReturningTouchdowns','PerfPointsAllowed',
  ]

if league == 'STS':
  league_scoring = {
  'PassingYards': 0.04,'PassingTouchdowns': 6,'PassingInterceptions': -2,
  'RushingYards': 0.1,'RushingTouchdowns': 6,'FumblesLost': -2,
  'ReceivingReceptions': 1,'ReceivingYards': 0.1,'ReceivingTouchdowns': 6,
  'ExtraPointMade':1,'ExtraPointMissed':-1,'FieldGoalMissed':-1,'FieldGoalMade':3,
  'DefenseReturningTouchdown':6,'DefenseTouchdown':6,'DefenseSack':1,'DefenseInterception':2,'DefenseFumbleRecovered':2,'DefenseSafety':2,'DefensePointsAllowed':-0.217391304348,'TacklesForLoss':0,'QuarterbackHits':0
  }
  roster_sizes = {'QB': 2,'RB': 3,'WR': 3,'TE': 1,'FLEX': 1,'DST': 1,'K': 1, 'BENCH': 6}
if league == 'IYNFYL':
  league_scoring = {
  'PassingYards': 0.04,'PassingTouchdowns': 6,'PassingInterceptions': -2,
  'RushingYards': 0.1,'RushingTouchdowns': 6,'FumblesLost': -2,
  'ReceivingReceptions': 1,'ReceivingYards': 0.1,'ReceivingTouchdowns': 6,
  'ExtraPointMade':1,'ExtraPointMissed':-1,'FieldGoalMissed':-1,'FieldGoalMade':3,
  'DefenseReturningTouchdown':6,'DefenseTouchdown':6,'DefenseSack':1,'DefenseInterception':4,'DefenseFumbleRecovered':4,'DefenseSafety':2,'DefensePointsAllowed':-0.217391304348,'TacklesForLoss':0,'QuarterbackHits':0
  }
  roster_sizes = {'QB': 1,'RB': 2,'WR': 3,'TE': 1,'FLEX': 2,'DST': 1,'K': 0, 'BENCH': 6}


relevant_ml_columns = [
  'ProjPassingYards',
  'ProjPassingTouchdowns', 'ProjPassingInterceptions', 'ProjRushingYards', 'ProjRushingTouchdowns',
  'ProjReceivingReceptions', 'ProjReceivingYards',
  'ProjReceivingTouchdowns', 'ProjFumblesLost',
  'ProjFieldGoalsMade', 'ProjFieldGoalsMissed', 'ProjExtraPointsMade',
  'ProjExtraPointsMissed', 'PerfLeaguePoints', 
  'ProjTacklesForLoss','ProjSacks','ProjQuarterbackHits','ProjInterceptions','ProjFumblesRecovered','ProjSafeties','ProjDefensiveTouchdowns','ProjPointsAllowed',
  'KvsARI', 'KvsATL', 'KvsBAL', 'KvsBUF', 'KvsCAR', 'KvsCHI', 'KvsCIN', 'KvsCLE', 'KvsDAL', 'KvsDEN', 'KvsDET', 'KvsGB', 'KvsHOU', 'KvsIND', 'KvsJAX', 'KvsKC', 'KvsLAC', 'KvsLAR', 'KvsLV', 'KvsMIA', 'KvsMIN', 'KvsNE', 'KvsNO', 'KvsNYG', 'KvsNYJ', 'KvsPHI', 'KvsPIT', 'KvsSEA', 'KvsSF', 'KvsTB', 'KvsTEN', 'KvsWAS', 
  'QBvsARI', 'QBvsATL', 'QBvsBAL', 'QBvsBUF', 'QBvsCAR', 'QBvsCHI', 'QBvsCIN', 'QBvsCLE', 'QBvsDAL', 'QBvsDEN', 'QBvsDET', 'QBvsGB', 'QBvsHOU', 'QBvsIND', 'QBvsJAX', 'QBvsKC', 'QBvsLAC', 'QBvsLAR', 'QBvsLV', 'QBvsMIA', 'QBvsMIN', 'QBvsNE', 'QBvsNO', 'QBvsNYG', 'QBvsNYJ', 'QBvsPHI', 'QBvsPIT', 'QBvsSEA', 'QBvsSF', 'QBvsTB', 'QBvsTEN', 'QBvsWAS', 
  'RBvsARI', 'RBvsATL', 'RBvsBAL', 'RBvsBUF', 'RBvsCAR', 'RBvsCHI', 'RBvsCIN', 'RBvsCLE', 'RBvsDAL', 'RBvsDEN', 'RBvsDET', 'RBvsGB', 'RBvsHOU', 'RBvsIND', 'RBvsJAX', 'RBvsKC', 'RBvsLAC', 'RBvsLAR', 'RBvsLV', 'RBvsMIA', 'RBvsMIN', 'RBvsNE', 'RBvsNO', 'RBvsNYG', 'RBvsNYJ', 'RBvsPHI', 'RBvsPIT', 'RBvsSEA', 'RBvsSF', 'RBvsTB', 'RBvsTEN', 'RBvsWAS', 
  'TEvsARI', 'TEvsATL', 'TEvsBAL', 'TEvsBUF', 'TEvsCAR', 'TEvsCHI', 'TEvsCIN', 'TEvsCLE', 'TEvsDAL', 'TEvsDEN', 'TEvsDET', 'TEvsGB', 'TEvsHOU', 'TEvsIND', 'TEvsJAX', 'TEvsKC', 'TEvsLAC', 'TEvsLAR', 'TEvsLV', 'TEvsMIA', 'TEvsMIN', 'TEvsNE', 'TEvsNO', 'TEvsNYG', 'TEvsNYJ', 'TEvsPHI', 'TEvsPIT', 'TEvsSEA', 'TEvsSF', 'TEvsTB', 'TEvsTEN', 'TEvsWAS', 
  'WRvsARI', 'WRvsATL', 'WRvsBAL', 'WRvsBUF', 'WRvsCAR', 'WRvsCHI', 'WRvsCIN', 'WRvsCLE', 'WRvsDAL', 'WRvsDEN', 'WRvsDET', 'WRvsGB', 'WRvsHOU', 'WRvsIND', 'WRvsJAX', 'WRvsKC', 'WRvsLAC', 'WRvsLAR', 'WRvsLV', 'WRvsMIA', 'WRvsMIN', 'WRvsNE', 'WRvsNO', 'WRvsNYG', 'WRvsNYJ', 'WRvsPHI', 'WRvsPIT', 'WRvsSEA', 'WRvsSF', 'WRvsTB', 'WRvsTEN', 'WRvsWAS', 
  'DSTvsARI', 'DSTvsATL', 'DSTvsBAL', 'DSTvsBUF', 'DSTvsCAR', 'DSTvsCHI', 'DSTvsCIN', 'DSTvsCLE', 'DSTvsDAL', 'DSTvsDEN', 'DSTvsDET', 'DSTvsGB', 'DSTvsHOU', 'DSTvsIND', 'DSTvsJAX', 'DSTvsKC', 'DSTvsLAC', 'DSTvsLAR', 'DSTvsLV', 'DSTvsMIA', 'DSTvsMIN', 'DSTvsNE', 'DSTvsNO', 'DSTvsNYG', 'DSTvsNYJ', 'DSTvsPHI', 'DSTvsPIT', 'DSTvsSEA', 'DSTvsSF', 'DSTvsTB', 'DSTvsTEN', 'DSTvsWAS', 
  'TimeWeight'
  ]

relevant_input_columns = [
  'ProjPassingYards',
  'ProjPassingTouchdowns', 'ProjPassingInterceptions', 'ProjRushingYards', 'ProjRushingTouchdowns',
  'ProjReceivingReceptions', 'ProjReceivingYards',
  'ProjReceivingTouchdowns', 'ProjFumblesLost',
  'ProjFieldGoalsMade', 'ProjFieldGoalsMissed', 'ProjExtraPointsMade',
  'ProjExtraPointsMissed',
  'ProjTacklesForLoss','ProjSacks','ProjQuarterbackHits','ProjInterceptions','ProjFumblesRecovered','ProjSafeties','ProjDefensiveTouchdowns','ProjPointsAllowed',
  'KvsARI', 'KvsATL', 'KvsBAL', 'KvsBUF', 'KvsCAR', 'KvsCHI', 'KvsCIN', 'KvsCLE', 'KvsDAL', 'KvsDEN', 'KvsDET', 'KvsGB', 'KvsHOU', 'KvsIND', 'KvsJAX', 'KvsKC', 'KvsLAC', 'KvsLAR', 'KvsLV', 'KvsMIA', 'KvsMIN', 'KvsNE', 'KvsNO', 'KvsNYG', 'KvsNYJ', 'KvsPHI', 'KvsPIT', 'KvsSEA', 'KvsSF', 'KvsTB', 'KvsTEN', 'KvsWAS', 
  'QBvsARI', 'QBvsATL', 'QBvsBAL', 'QBvsBUF', 'QBvsCAR', 'QBvsCHI', 'QBvsCIN', 'QBvsCLE', 'QBvsDAL', 'QBvsDEN', 'QBvsDET', 'QBvsGB', 'QBvsHOU', 'QBvsIND', 'QBvsJAX', 'QBvsKC', 'QBvsLAC', 'QBvsLAR', 'QBvsLV', 'QBvsMIA', 'QBvsMIN', 'QBvsNE', 'QBvsNO', 'QBvsNYG', 'QBvsNYJ', 'QBvsPHI', 'QBvsPIT', 'QBvsSEA', 'QBvsSF', 'QBvsTB', 'QBvsTEN', 'QBvsWAS', 
  'RBvsARI', 'RBvsATL', 'RBvsBAL', 'RBvsBUF', 'RBvsCAR', 'RBvsCHI', 'RBvsCIN', 'RBvsCLE', 'RBvsDAL', 'RBvsDEN', 'RBvsDET', 'RBvsGB', 'RBvsHOU', 'RBvsIND', 'RBvsJAX', 'RBvsKC', 'RBvsLAC', 'RBvsLAR', 'RBvsLV', 'RBvsMIA', 'RBvsMIN', 'RBvsNE', 'RBvsNO', 'RBvsNYG', 'RBvsNYJ', 'RBvsPHI', 'RBvsPIT', 'RBvsSEA', 'RBvsSF', 'RBvsTB', 'RBvsTEN', 'RBvsWAS', 
  'TEvsARI', 'TEvsATL', 'TEvsBAL', 'TEvsBUF', 'TEvsCAR', 'TEvsCHI', 'TEvsCIN', 'TEvsCLE', 'TEvsDAL', 'TEvsDEN', 'TEvsDET', 'TEvsGB', 'TEvsHOU', 'TEvsIND', 'TEvsJAX', 'TEvsKC', 'TEvsLAC', 'TEvsLAR', 'TEvsLV', 'TEvsMIA', 'TEvsMIN', 'TEvsNE', 'TEvsNO', 'TEvsNYG', 'TEvsNYJ', 'TEvsPHI', 'TEvsPIT', 'TEvsSEA', 'TEvsSF', 'TEvsTB', 'TEvsTEN', 'TEvsWAS', 
  'WRvsARI', 'WRvsATL', 'WRvsBAL', 'WRvsBUF', 'WRvsCAR', 'WRvsCHI', 'WRvsCIN', 'WRvsCLE', 'WRvsDAL', 'WRvsDEN', 'WRvsDET', 'WRvsGB', 'WRvsHOU', 'WRvsIND', 'WRvsJAX', 'WRvsKC', 'WRvsLAC', 'WRvsLAR', 'WRvsLV', 'WRvsMIA', 'WRvsMIN', 'WRvsNE', 'WRvsNO', 'WRvsNYG', 'WRvsNYJ', 'WRvsPHI', 'WRvsPIT', 'WRvsSEA', 'WRvsSF', 'WRvsTB', 'WRvsTEN', 'WRvsWAS', 
  'DSTvsARI', 'DSTvsATL', 'DSTvsBAL', 'DSTvsBUF', 'DSTvsCAR', 'DSTvsCHI', 'DSTvsCIN', 'DSTvsCLE', 'DSTvsDAL', 'DSTvsDEN', 'DSTvsDET', 'DSTvsGB', 'DSTvsHOU', 'DSTvsIND', 'DSTvsJAX', 'DSTvsKC', 'DSTvsLAC', 'DSTvsLAR', 'DSTvsLV', 'DSTvsMIA', 'DSTvsMIN', 'DSTvsNE', 'DSTvsNO', 'DSTvsNYG', 'DSTvsNYJ', 'DSTvsPHI', 'DSTvsPIT', 'DSTvsSEA', 'DSTvsSF', 'DSTvsTB', 'DSTvsTEN', 'DSTvsWAS', 
  ]


In [ ]:
#@title Define projected_df

projections_df = pd.read_csv('/content/drive/MyDrive/FantasyFootballCoding/Sources/HistoricalWeeklyProjectionSource.csv').iloc[:, 1:]
projections_df = projections_df[projections_df['player'].notna()]


projections_df['duplicate'] = projections_df.duplicated(subset=['player','Year','Week'],keep='last')
projections_df = projections_df[projections_df.duplicate != True].drop(columns={'duplicate'})

df = pd.DataFrame()
for p in list(proj_column_dict.keys()):
  x = projections_df.loc[projections_df['Position'] == p]
  x[proj_column_dict[p]] = x.projstats.str.split(' ',expand=True)
  x = x.drop(columns={'projstats'})
  df = df.append(x, ignore_index=True)
projections_df = df.fillna(0)

In [ ]:
#@title Define performance_df

performance_df = pd.read_csv('/content/drive/MyDrive/FantasyFootballCoding/Sources/HistoricalWeeklyPerformanceSource.csv').iloc[:, 1:]
performance_df = performance_df[performance_df['player'].notna()]

performance_df['duplicate'] = performance_df.duplicated(subset=['player','Year','Week'],keep='last')
performance_df = performance_df[performance_df.duplicate != True].drop(columns={'duplicate'})

df = pd.DataFrame()
for p in list(perf_column_dict.keys()):
  x = performance_df.loc[performance_df['Position'] == p]
  x[perf_column_dict[p]] = x.stats.str.split(' ',expand=True)
  x = x.drop(columns={'stats'})
  df = df.append(x, ignore_index=True)
performance_df = df.fillna(0)

In [ ]:
#@title Define source_df
source_df = pd.merge(projections_df, performance_df, on=["player", "Year", "Week",'Position','PlayerTeam','PlayerPosition','OppTeam'],how='left')
source_df = source_df.dropna()

In [ ]:
#@title Define data_df

data_df = source_df[relevant_data_columns].copy()
data_df['ProjFieldGoalsMissed'] = data_df['ProjFieldGoalAttempts'].astype(float) - data_df['ProjFieldGoalsMade'].astype(float)
data_df['ProjExtraPointsMissed'] = data_df['ProjExtraPointAttempts'].astype(float) - data_df['ProjExtraPointsMade'].astype(float)
data_df['PerfFieldGoalsMissed'] = data_df['PerfFieldGoalAttempts'].astype(float) - data_df['PerfFieldGoalsMade'].astype(float)
data_df['PerfExtraPointsMissed'] = data_df['PerfExtraPointAttempts'].astype(float) - data_df['PerfExtraPointsMade'].astype(float)
data_df.drop(columns={'ProjFieldGoalAttempts','ProjExtraPointAttempts','PerfExtraPointAttempts','PerfFieldGoalAttempts'})

s = data_df['PlayerPosition']
data_df = data_df.join(pd.get_dummies(s))

data_df['PerfLeaguePoints'] = (
       ((data_df['PerfPassingYards']).astype(float))*league_scoring['PassingYards'] + ((data_df['PerfPassingTouchdowns']).astype(float))*league_scoring['PassingTouchdowns'] + ((data_df['PerfPassingInterceptions']).astype(float))*league_scoring['PassingInterceptions'] +
       ((data_df['PerfRushingYards']).astype(float))*league_scoring['RushingYards'] + ((data_df['PerfRushingTouchdowns']).astype(float))*league_scoring['RushingTouchdowns'] + ((data_df['PerfFumblesLost']).astype(float))*league_scoring['FumblesLost'] + 
       ((data_df['PerfReceivingReceptions']).astype(float))*league_scoring['ReceivingReceptions'] + ((data_df['PerfReceivingYards']).astype(float))*league_scoring['ReceivingYards'] + ((data_df['PerfReceivingTouchdowns']).astype(float))*league_scoring['ReceivingTouchdowns'] + 
       ((data_df['PerfFieldGoalsMade']).astype(float))*league_scoring['FieldGoalMade'] + ((data_df['PerfFieldGoalsMissed']).astype(float))*league_scoring['FieldGoalMissed'] + ((data_df['PerfExtraPointsMade']).astype(float))*league_scoring['ExtraPointMade'] + ((data_df['PerfExtraPointsMissed']).astype(float))*league_scoring['ExtraPointMissed'] +
       ((data_df['PerfTacklesForLoss']).astype(float))*league_scoring['TacklesForLoss'] + ((data_df['PerfSacks']).astype(float))*league_scoring['DefenseSack'] + ((data_df['PerfQuarterbackHits']).astype(float))*league_scoring['QuarterbackHits'] + ((data_df['PerfInterceptions']).astype(float))*league_scoring['DefenseInterception'] + ((data_df['PerfFumblesRecovered']).astype(float))*league_scoring['DefenseFumbleRecovered'] + ((data_df['PerfSafeties']).astype(float))*league_scoring['DefenseSafety'] + ((data_df['PerfDefensiveTouchdowns']).astype(float))*league_scoring['DefenseTouchdown'] + ((data_df['PerfReturningTouchdowns']).astype(float))*league_scoring['DefenseTouchdown'] + ((((data_df['PerfPointsAllowed']).astype(float))*league_scoring['DefensePointsAllowed']) + data_df['DST']*5)
       )

data_df["PlayerPosition"].replace({"FB": "RB"}, inplace=True)

data_df['matchup'] = data_df['PlayerPosition'].astype(str) + 'vs' + data_df['OppTeam'].astype(str)
s = data_df['matchup']
data_df = data_df.join(pd.get_dummies(s))

data_df['TimeWeight'] = data_df['Year'] - 2012
data_df['Week'] = data_df['Week'].astype(int)
data_df['TimeWeight'] = (data_df['Week']) + ((data_df['TimeWeight'] - 1).multiply(18))
min_TimeWeight = 0
max_TimeWeight = data_df['TimeWeight'].max()
TimeWeight_range = max_TimeWeight - min_TimeWeight
data_df['TimeWeight'] = data_df['TimeWeight'].apply(lambda x: (x - min_TimeWeight) / TimeWeight_range)
data_df['TimeWeight'] = data_df['TimeWeight'].apply(lambda x: (x*4) ** 2)

In [ ]:
#@title Define train_ml_df

train_ml_df = data_df[relevant_ml_columns].copy()

In [ ]:
#@title Define regr.fit

X = train_ml_df.drop(columns={'PerfLeaguePoints','TimeWeight'})
y = train_ml_df['PerfLeaguePoints']
n = train_ml_df['TimeWeight']

regr = linear_model.LinearRegression(fit_intercept=False)
regr.fit(X, y, n)

In [ ]:
#@title Define scrape_pos_year_week

scrape_df = pd.DataFrame()

def scrape_pos_year_week(pos,year,week,t):
  url = f"https://fantasydata.com/nfl/fantasy-football-weekly-projections?position={pos}&season={year}&seasontype=1&scope=2&startweek={week}&endweek={week}"

  global scrape_df
  
  try:
    driver.get(url)
  except TimeOutException:
    Pass

  driver.find_element(By.XPATH, "//a[@ng-click='SetPageSize(300);']").click()

  time.sleep(t)

  players = driver.find_element(By.CSS_SELECTOR, "body > div.fd-body.ng-scope > div > div.fd-content > div.container.page-container > div > div.tab-section-container.horizontal > div > div > div.tab-content > section > div.ng-scope > div.stats-grid-container > div.k-grid.k-widget.k-display-block.k-grid-lockedcolumns > div.k-grid-content-locked > table > tbody")
  player_list = players.text.split('\n')
  player_list = player_list[::2]
  projstats = driver.find_element(By.XPATH, "//tbody[@role='rowgroup']")
  projstats_list = projstats.text.split('\n')
  scrape_df = pd.DataFrame(
    {'player': player_list,
     'projstats': projstats_list
    })

  scrape_df['Position'] = f'{pos}'
  scrape_df['Year'] = f'{year}'
  scrape_df['Week'] = f'{week}'
  return scrape_df

In [ ]:
#@title Login to Website

driver.get('https://fantasydata.com/user/login')
email = driver.find_element(By.ID,'Email')
password = driver.find_element(By.ID,'Password')
email.send_keys(inputemail)
password.send_keys(inputpassword)
driver.find_element(By.XPATH, '//button[@class="btn fd-btn fd-btn-blue"]').click()

In [ ]:
#@title Scrape Website

pos_list = list(range(2,8,1))

df = pd.DataFrame()
for p in pos_list:
  t = 1.5
  scrape_pos_year_week(p,year,week,t)
  while len(scrape_df.index) == 1:
    t = t+1
    print('retrying '+f'{year}'+','+f'{week}'+','+f'{p}'f' {t}s')
    scrape_pos_year_week(p,year,week,t)
  scrape_df['player'] = scrape_df['player'].dropna().map(lambda x: ''.join(x.split(' ')[1:3]))
  df = df.append(scrape_df, ignore_index=True)
  print(f'{year}'+','+f'{week}'+','+f'{p}')

upcoming_projections_df = df

In [ ]:
#@title Refine upcoming_projections_df

df = pd.DataFrame()
for p in list(proj_column_dict.keys()):
  x = upcoming_projections_df.loc[upcoming_projections_df['Position'] == f'{p}']
  x[proj_column_dict[p]] = x.projstats.str.split(' ',expand=True)
  x = x.drop(columns={'projstats'})
  df = df.append(x, ignore_index=True)
upcoming_projections_df = df.fillna(0)

upcoming_projections_df['ProjFieldGoalsMissed'] = upcoming_projections_df['ProjFieldGoalAttempts'].astype(float) - upcoming_projections_df['ProjFieldGoalsMade'].astype(float)
upcoming_projections_df['ProjExtraPointsMissed'] = upcoming_projections_df['ProjExtraPointAttempts'].astype(float) - upcoming_projections_df['ProjExtraPointsMade'].astype(float)
upcoming_projections_df = upcoming_projections_df.drop(columns={'ProjFieldGoalAttempts','ProjExtraPointAttempts'})

upcoming_projections_df["PlayerPosition"].replace({"FB": "RB"}, inplace=True)

upcoming_projections_df['matchup'] = upcoming_projections_df['PlayerPosition'] + 'vs' + upcoming_projections_df['OppTeam']
s = upcoming_projections_df['matchup']
upcoming_projections_df = upcoming_projections_df.join(pd.get_dummies(s))

In [ ]:
#@title Define DFs for ML

upcoming_ids = upcoming_projections_df[['player','Year','Week']]
upcoming_data = upcoming_projections_df[relevant_input_columns]

In [ ]:
#@title Input Data for ML

upcoming_ids['ModelLeaguePoints'] = regr.predict(upcoming_data)

In [ ]:
#@title Model Projected League Points

# upcoming_ids.sort_values(by='ModelLeaguePoints', ascending=False)

In [ ]:
#@title League Definitions
if league == 'IYNFYL':
  league_id =  #your league_id
  swid = '' #your swid
  espn_s2 = '' #your espn_s2
  roster_sizes = {'QB': 1,'RB': 2,'WR': 3,'TE': 1,'FLEX': 2,'DST': 1,'K': 0, 'BENCH': 6}
  team = 'roster0'

if league == 'STS':
  league_id =  #your league_id
  swid = '' #your swid
  espn_s2 = '' #your espn_s2
  roster_sizes = {'QB': 2,'RB': 3,'WR': 3,'TE': 1,'FLEX': 1,'DST': 1,'K': 1, 'BENCH': 6}
  team = 'roster3'

league = League(league_id, year, espn_s2, swid)

In [ ]:
#@title Create Rosters Dataframe
rosters = {}

for x in range(len(league.teams)):
    rosters['roster{0}'.format(x)] = (league.teams[x].__dict__)['roster']

roster_df = pd.DataFrame.from_dict(rosters, orient='index')
roster_df = roster_df.transpose()

roster_df = roster_df.astype(str)
roster_df = roster_df.replace({'Player\(':''}, regex=True)
roster_df = roster_df.replace({"\)":''}, regex=True)

roster_df = roster_df.stack().reset_index()
roster_df['Player'] = roster_df[0]
roster_df['FantasyTeam'] = roster_df['level_1']
roster_df = roster_df[roster_df.Player != 'None']
roster_df = roster_df[['Player', 'FantasyTeam']]
roster_df['Player'] = roster_df['Player'].map(lambda x: ''.join(x.split(' ')[:2]))

In [ ]:
#@title Define upcoming_ids
upcoming_ids['Player'] = upcoming_ids['player']
upcoming_ids = upcoming_ids.drop(columns={'player'})
df = upcoming_ids.join(roster_df.set_index('Player'),on='Player',how='left',rsuffix='_y')
df = df[['Player', 'ModelLeaguePoints', 'FantasyTeam']]
pos_df = source_df[['player','PlayerPosition']]
pos_df['duplicate'] = pos_df.duplicated(subset=['player','PlayerPosition'],keep='last')
pos_df = pos_df[pos_df.duplicate != True].drop(columns={'duplicate'})
pos_df['Player'] = pos_df['player']
pos_df = pos_df.drop(columns={'player'})
df = pd.merge(pos_df, df, on=['Player']).fillna('None')

In [ ]:
#@title
league.teams

In [ ]:
#@title
team = 'roster0'

In [ ]:
#@title Define Positional Dataframes
qb_df = df.loc[df['FantasyTeam'].eq('None') | df['FantasyTeam'].eq(team)].loc[df['PlayerPosition'].eq('QB')].sort_values(by='ModelLeaguePoints', ascending=False).head(roster_sizes['QB'])
rb_df = df.loc[df['FantasyTeam'].eq('None') | df['FantasyTeam'].eq(team)].loc[df['PlayerPosition'].eq('RB')].sort_values(by='ModelLeaguePoints', ascending=False).head(roster_sizes['RB'])
wr_df = df.loc[df['FantasyTeam'].eq('None') | df['FantasyTeam'].eq(team)].loc[df['PlayerPosition'].eq('WR')].sort_values(by='ModelLeaguePoints', ascending=False).head(roster_sizes['WR'])
te_df = df.loc[df['FantasyTeam'].eq('None') | df['FantasyTeam'].eq(team)].loc[df['PlayerPosition'].eq('TE')].sort_values(by='ModelLeaguePoints', ascending=False).head(roster_sizes['TE'])
flex_df = df.loc[df['FantasyTeam'].eq('None') | df['FantasyTeam'].eq(team)].loc[df['PlayerPosition'].eq('RB') | df['PlayerPosition'].eq('WR') | df['PlayerPosition'].eq('TE')].sort_values(by='ModelLeaguePoints', ascending=False).head(roster_sizes['WR'] + roster_sizes['RB'] + roster_sizes['TE'] + 5)
k_df = df.loc[df['FantasyTeam'].eq('None') | df['FantasyTeam'].eq(team)].loc[df['PlayerPosition'].eq('K')].sort_values(by='ModelLeaguePoints', ascending=False).head(roster_sizes['K'])
dst_df = df.loc[df['PlayerPosition'].eq('DST')].sort_values(by='ModelLeaguePoints', ascending=False)

In [ ]:
#@title Display All Positions
display_side_by_side(qb_df,rb_df,wr_df,te_df,flex_df,k_df,dst_df,titles=['QBs','RBs','WRs','TEs','Flex','K','DST'])

In [ ]:
#@title ML Coefficients

pd.DataFrame({'Column':list(upcoming_data.columns),'coef':list(regr.coef_)}).sort_values(by='coef')

In [ ]:
#@title
